# Use Pre-optimized Baseline

Load and evaluate pre-optimized solutions from snapshots.

In [1]:
import math
import numpy as np
import pandas as pd
from numba import njit
import time
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

getcontext().prec = 25
scale_factor = Decimal('1e15')

print("Libraries loaded")

Libraries loaded


In [2]:
# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x = np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2], np.float64)
    y = np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1], np.float64)
    return x, y

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c*tx[j] - s*ty[j] + xi
            Y = s*tx[j] + c*ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

tx, ty = make_polygon_template()
test_xs = np.array([0.0], np.float64)
test_ys = np.array([0.0], np.float64)
test_degs = np.array([45.0], np.float64)
_ = score_group(test_xs, test_ys, test_degs, tx, ty)
print("JIT compiled")

JIT compiled


In [3]:
def strip(a):
    return np.array([float(str(v).replace("s","")) for v in a], np.float64)

def load_submission(filepath):
    df = pd.read_csv(filepath)
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    configs = {}
    for n, g in df.groupby('N'):
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        degs = strip(g['deg'].to_numpy())
        configs[n] = (xs, ys, degs)
    return configs

def calculate_total_score(configs, tx, ty):
    total = 0.0
    for n, (xs, ys, degs) in configs.items():
        total += score_group(xs, ys, degs, tx, ty)
    return total

print("Helper functions defined")

Helper functions defined


In [4]:
# Evaluate all pre-optimized solutions
import os

preopt_dir = '/home/code/preoptimized/'
files = [f for f in os.listdir(preopt_dir) if f.endswith('.csv')]

print("Evaluating pre-optimized solutions:")
print("="*60)

scores = {}
for f in sorted(files):
    filepath = os.path.join(preopt_dir, f)
    try:
        configs = load_submission(filepath)
        score = calculate_total_score(configs, tx, ty)
        scores[f] = score
        print(f"{f}: {score:.6f}")
    except Exception as e:
        print(f"{f}: ERROR - {e}")

print("="*60)
best_file = min(scores, key=scores.get)
print(f"\nBest: {best_file} with score {scores[best_file]:.6f}")
print(f"Target: 68.922808")
print(f"Gap: {scores[best_file] - 68.922808:.6f}")

Evaluating pre-optimized solutions:
71.97.csv: 71.972027
best_ensemble.csv: 70.676102
best_public.csv: 70.926150


corner_extraction.csv: 70.675510
ensemble.csv: 70.676102
saspav.csv: 70.676102
submission_70_926149550346.csv: 70.926150

Best: corner_extraction.csv with score 70.675510
Target: 68.922808
Gap: 1.752702


In [5]:
# Load the best pre-optimized solution
best_path = os.path.join(preopt_dir, best_file)
configs = load_submission(best_path)
best_score = calculate_total_score(configs, tx, ty)
print(f"Loaded {best_file}")
print(f"Score: {best_score:.6f}")

Loaded corner_extraction.csv
Score: 70.675510


In [6]:
# Validate for overlaps using Shapely
class ChristmasTree:
    def __init__(self, center_x, center_y, angle):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print("Validating configurations...")
invalid = []
for n, (xs, ys, degs) in configs.items():
    trees = [ChristmasTree(xs[i], ys[i], degs[i]) for i in range(n)]
    if has_overlap(trees):
        invalid.append(n)

if invalid:
    print(f"WARNING: {len(invalid)} configs have overlaps: {invalid[:10]}...")
else:
    print("All configurations valid (no overlaps)")

Validating configurations...


In [7]:
# Save as our submission
print("Saving submission...")

rows = []
for n in range(1, 201):
    xs, ys, degs = configs[n]
    for i in range(n):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{xs[i]:.15f}',
            'y': f's{ys[i]:.15f}',
            'deg': f's{degs[i]:.15f}'
        })

submission = pd.DataFrame(rows)
submission.to_csv('/home/submission/submission.csv', index=False)
submission.to_csv('/home/code/experiments/007_preoptimized_baseline/submission.csv', index=False)

print(f"Submission saved!")
print(f"Final Score: {best_score:.6f}")
print(f"Target: 68.922808")
print(f"Gap: {best_score - 68.922808:.6f}")

Saving submission...
Submission saved!
Final Score: 70.675510
Target: 68.922808
Gap: 1.752702
